In [11]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def jet_spinode(states):

    dt = 0.01
    
    states = np.array(states)




In [3]:
##Wavenumbers
k = 12 * np.pi
m = 6 * np.pi
m_U = 14 * np.pi #vertical wavenumber
##

r_m = 0.1
N0_squared = 100


k_e = np.sqrt(k**2 + m**2)
k_plus = np.sqrt(k**2 +(m+m_U)**2)

W_e = np.array([[-1, k / (k_e**2)],
                [-k * N0_squared, -1]])
W_plus = np.array([[-1, -k / (k_plus**2)],
                [k * N0_squared, -1]])

W = np.block([[W_e, np.zeros((2, 2))],
             [np.zeros((2, 2)), W_plus]])

L_e = np.array([[-(k*(k_plus**2 - m_U**2)) / (2 * (k_e**2)), 0],
                [0, k / 2]])

L_plus = np.array([[-(k*(m_U**2 - k_e**2)) / (2 * (k_plus**2)), 0],
                [0, -k / 2]])

L = np.block([[np.zeros((2, 2)), L_e],
             [L_plus, np.zeros((2, 2))]])

In [4]:
state_ini = np.array([0.0, 0.0, 0.0, 0.0, 0.001])

dt = 0.01
    
states = state_ini[0:4]
U = state_ini[4]

xkw = np.random.normal(0, 1)
epsilon = 0.01

#diffusion coefficient
g2s = (2 * np.sqrt(2) / k_e) * np.sqrt(epsilon)
g2U = 0.0

#drift coeffificent
g1s = (W @ states) + (U * (L @ states))
g1U = ((0.25 * k * (k_plus**2 - k_e**2) * states[0] * states[2])) - (r_m * U)

#predict forward dynamics
sp1 = states + (g1s * dt) + (g2s * np.sqrt(dt) * xkw)
Up1 = U + (g1U * dt) 


In [14]:
def jet_spinode(states):

    dt = 0.01
    
    states = state_ini[0:4]
    U = state_ini[4]

    epsilon = 0.01

    #diffusion coefficient
    g2s = (2 * np.sqrt(2) / k_e) * np.sqrt(epsilon) * np.random.normal(0, 1)
    g2s_m = np.array([g2s, 0.0, 0.0, 0.0])
    g2U = 0.0

    #drift coeffificent
    g1s = (W @ states) + (U * (L @ states))
    g1U = ((0.25 * k * (k_plus**2 - k_e**2) * states[0] * states[2])) - (r_m * U)

    #predict forward dynamics
    sp1 = states + (g1s * dt) + (g2s_m * np.sqrt(dt))
    Up1 = U + (g1U * dt) 

    g1 = np.append(g1s, g1U)
    g2 = np.append(g2s, g2U)

    return [np.asarray([sp1, Up1]), 
            np.asarray([g1]), 
            np.asarray([g2])]


In [15]:
def generate_data(num_samples, num_time_steps, dt):
    # Preallocate arrays to store states for each time step
    states = np.empty((num_samples, num_time_steps, 5))
    drifts = np.empty((num_samples, num_time_steps, 5))
    diffusions = np.empty((num_samples, num_time_steps, 5))

    # Loop over each sample
    for i in range(num_samples):
        # Initialize the state
        state_ini = np.array([0, 0, 0, 0, 0.001])

        # Loop over each time step
        for t in range(num_time_steps):
            # Update the state using the SDE
            state, drift, diffusion = jet_spinode(state_ini)
            # Store the state, drift, and diffusion
            states[i, t] = state
            drifts[i, t] = drift
            diffusions[i, t] = diffusion
            # Update the initial state for the next time step
            state_ini = state

    return states, drifts, diffusions

# Use the function to generate data
num_samples = 1000
num_time_steps = 100
dt = 0.01
states, drifts, diffusions = generate_data(num_samples, num_time_steps, dt)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.